# le module make_date

In [33]:
import os
import sys
src_dir = os.path.abspath('../src')
sys.path.append(src_dir)



In [34]:
from data  import make_model_raw_Data

In [35]:
df1,df2=make_model_raw_Data.get_data(bucket_name="odione")

 Pour le dataset : Firstname 

shape (6946, 3) 

 Pour le dataset : Transcriptions 

shape (241, 4)




In [7]:
df1.head(2)

,firstname,male,female
0,marie,10145,2390322
1,jean,1869615,6476


In [8]:
df2.head(2)

,subject_line,groundtruth,prediction,sex
0,ebb26ada-044c-4c62-9dbc-a9c8d505d31c,surname: Chardon firstname: Marie occupation: ...,nom: Chardon prénom: Marie date_naissance: 30 ...,femme
1,338496f5-e4ca-43ac-aa5c-429cb3f6ac00,surname: Lhopital firstname: Louis-Jean occupa...,nom: Lhopital prénom: Louis Jean date_naissanc...,homme


# le module buid_features

In [43]:
from features import make_features_data

In [44]:
(x_train,y_train),(x_test,y_test)=make_features_data.preprocess_data(df1,df2)

 Train : (192,) (192,)
 Test : (49,) (49,)
well done


In [45]:
x_train.head(2)

prenom
etienne      [tensor(101), tensor(2053), tensor(2213), tens...
alexandre    [tensor(101), tensor(2053), tensor(2213), tens...
Name: input_ids, dtype: object

In [46]:
y_train.head(2)

prenom
etienne      1
alexandre    1
Name: labels, dtype: int64

In [47]:
df2.head(3)

,subject_line,groundtruth,prediction,sex
0,ebb26ada-044c-4c62-9dbc-a9c8d505d31c,surname: Chardon firstname: Marie occupation: ...,nom: Chardon prénom: Marie date_naissance: 30 ...,femme
1,338496f5-e4ca-43ac-aa5c-429cb3f6ac00,surname: Lhopital firstname: Louis-Jean occupa...,nom: Lhopital prénom: Louis Jean date_naissanc...,homme
2,e6a279da-9b6f-4f49-b498-64857bc50d1e,surname: Papin firstname: Marie occupation: id...,nom: Pyrin prénom: Marie date_naissance: 55 re...,femme


In [5]:
make_fetaure17.upload_preproces_in_online_Bucket()

Une erreur s'est produite lors de l'envoi du fichier : Failed to upload /home/onyxia/work/ProjetNlpavril2024/Project1_odione/classification_by_LLM_DL/data/df_after_tokenizeation.csv to odione/Socface_Data/df_after_tokenizeation.csv: An error occurred (InvalidTokenId) when calling the PutObject operation: The security token included in the request is invalid


# Train a simple and specific model

In [49]:
from models import train_single_specific_model as train_one

In [50]:
model,performanence=train_one.training(15,10,5)

 Pour le dataset : Firstname 

shape (6946, 3) 

 Pour le dataset : Transcriptions 

shape (241, 4)


 Train : (192,) (192,)
 Test : (49,) (49,)
well done
Epoch [100/100], Accuracy: 0.8168229166666666


In [51]:
# Afficher le résumé du modèle
from torchsummary import summary  # Importer la fonction summary de torchsummary

summary(model, input_size=(1, model.input_len))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 15]           1,065
            Linear-2                [-1, 1, 10]             160
            Linear-3                 [-1, 1, 5]              55
            Linear-4                 [-1, 1, 2]              12
Total params: 1,292
Trainable params: 1,292
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


# Trainnign by Cross validation

In [58]:
from models import  train_model_Cros_val as cv

In [59]:
wrapper_model=cv.PyTorchModel(l1=10,l2=10,l3=10)

In [60]:
type(wrapper_model)

models.train_model_Cros_val.PyTorchModel

In [84]:
x_train.values

tensor([  101,  2053,  2213,  1024,  2176,  2100,  1010,  3653,  3630,  2213,
         1024,  3744,  1011,  5292, 13876, 27870,  1010,  3058,  1035,  6583,
        21205,  5897,  1024,  2654,  1010, 22470,  2595,  1035,  6583, 21205,
         5897,  1024, 10026,  1010,  7189,  1024, 18178, 10020,  2139,  1042,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [61]:
wrapper_model.fit(x_train,y_train)

PyTorchModel(l1=10, l2=10, l3=10)

In [62]:
wrapper_model.score(x_train,y_train)

0.7135416666666666

In [63]:
params={"l1":[10],"l2":[10,20],"l3":[10]}
make_gridCh=cv.make_grid_search(x_train,y_train,grid_params=params,n_cv=2)

In [64]:
make_gridCh.best_params

{'l1': 10, 'l2': 10, 'l3': 10}

In [65]:
make_gridCh.best_score

0.59375

In [66]:
make_gridCh.best_model

PyTorchModel(l1=10, l2=10, l3=10)

In [67]:
make_gridCh.show_results()

Best Hyperparameters:  {'l1': 10, 'l2': 10, 'l3': 10} 

Best Score:  0.59375


# make predictions 

In [72]:
from models import  make_predicc as predictor

In [73]:
" set filesystem variables"
src_dir=os.path.abspath("../src")
sys.path.append(src_dir)
projet_path = os.path.abspath(os.path.join(src_dir, os.pardir))
sys.path.append(projet_path)
config_path = projet_path+'/config.yaml'

In [74]:
import yaml
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
best_model_config_filename="best_model_config.pth"
best_model_filename="best_model.pth"

file_path_in_s3 = config['data_for_prediction']
file_name_in_local="data_for_prediction.csv"

In [75]:
predicted_data_in_local="a changer"
predicted_data=predictor.predict(file_path_in_s3,file_name_in_local,predicted_data_in_local,best_model_filename,best_model_config_filename)

 Pour le dataset :  data_for_prediction.csv 

shape (10, 5) 

data_for_prediction  :  (10, 2)
well done
Model loaded successfully
Prediction succeful!


In [76]:
predicted_data

array([1, 1, 1, 1, 0, 1, 0, 1, 0, 0])